In [61]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline

In [2]:
spark = SparkSession.builder.appName('PC2').getOrCreate()

In [3]:
spark

## Importando Datos con PySpark

In [8]:
df = spark.read.csv('ECommerceChurn.csv',header=True,inferSchema=True)

In [9]:
df.limit(5).toPandas()

,CustomerID,Churn,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
0,50001,1,4.0,Mobile Phone,3,6.0,Debit Card,Female,3.0,3,Laptop & Accessory,2,Single,9,1,11.0,1.0,1.0,5.0,160
1,50002,1,NaN,Phone,1,8.0,UPI,Male,3.0,4,Mobile,3,Single,7,1,15.0,0.0,1.0,0.0,121
2,50003,1,NaN,Phone,1,30.0,Debit Card,Male,2.0,4,Mobile,3,Single,6,1,14.0,0.0,1.0,3.0,120
3,50004,1,0.0,Phone,3,15.0,Debit Card,Male,2.0,4,Laptop & Accessory,5,Single,8,0,23.0,0.0,1.0,3.0,134
4,50005,1,0.0,Phone,1,12.0,CC,Male,NaN,3,Mobile,5,Single,3,0,11.0,1.0,1.0,3.0,130


In [15]:
df.dtypes      # los tipos de datos de las variables están correctos, no es necesario realizar algún cambio

[('CustomerID', 'int'),
 ('Churn', 'int'),
 ('Tenure', 'double'),
 ('PreferredLoginDevice', 'string'),
 ('CityTier', 'int'),
 ('WarehouseToHome', 'double'),
 ('PreferredPaymentMode', 'string'),
 ('Gender', 'string'),
 ('HourSpendOnApp', 'double'),
 ('NumberOfDeviceRegistered', 'int'),
 ('PreferedOrderCat', 'string'),
 ('SatisfactionScore', 'int'),
 ('MaritalStatus', 'string'),
 ('NumberOfAddress', 'int'),
 ('Complain', 'int'),
 ('OrderAmountHikeFromlastYear', 'double'),
 ('CouponUsed', 'double'),
 ('OrderCount', 'double'),
 ('DaySinceLastOrder', 'double'),
 ('CashbackAmount', 'int')]

In [10]:
# Pregunta 1

In [16]:
#Remover NA
df = df.na.drop()

In [43]:
var_cat = [nC for nC,dt in df.dtypes if dt =='string']
var_num = [nC for nC,dt in df.dtypes if dt in ['int','double']]

In [44]:
var_num.remove('CustomerID')
var_num.remove('Churn')   # Variable Y

In [45]:
df[var_num].describe().toPandas()    # Reporte estadístico básico de todas las variables numéricas (no incluye var respuesta)

,summary,Tenure,CityTier,WarehouseToHome,HourSpendOnApp,NumberOfDeviceRegistered,SatisfactionScore,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
0,count,3774,3774,3774,3774,3774,3774,3774,3774,3774,3774,3774,3774,3774
1,mean,8.776894541600424,1.7077371489136195,15.743773184949655,2.9811870694223637,3.754107048224695,3.0564387917329094,4.216481187069422,0.2821939586645469,15.727609962904081,1.719660837307896,2.825384207737149,4.526232114467408,164.2151563328034
2,stddev,7.678508136323304,0.9367251653058004,8.594329775245635,0.7221577868103932,1.0252331082505033,1.3897297116506901,2.562142316392126,0.4501268879096836,3.6299101675533394,1.6745076665170657,2.4675969629393997,3.3713099811171667,24.089820509405968
3,min,0.0,1,5.0,0.0,1,1,1,0,11.0,0.0,1.0,0.0,0
4,max,51.0,3,127.0,5.0,6,5,22,1,26.0,16.0,16.0,46.0,325


In [41]:
df.groupBy("Churn").count().toPandas()     # Reporte de datos por clase

,Churn,count
0,1,631
1,0,3143


In [56]:
# Pregunta 2   (Modelo LR con ensamble solo de num)
lista_etapas = []

# Tratamiento con la variable respuesta
strIdx = StringIndexer(inputCol='Churn',outputCol='Y')
lista_etapas.append(strIdx)

# Ensamblador solo de variables numéricas
columnasVectores =  var_num
ensamblador = VectorAssembler(inputCols=columnasVectores,outputCol='X')
lista_etapas.append(ensamblador)

procesadorEtapas = Pipeline(stages=lista_etapas)
modelo = procesadorEtapas.fit(df)
df2 = modelo.transform(df)

# Construyendo el modelo de regresión logística
train, test = df2.randomSplit([0.8, 0.2], seed = 10)
lr = LogisticRegression(featuresCol = 'X', labelCol = 'Y')
lrModel = lr.fit(train)
predictions = lrModel.transform(test)
evaluator = BinaryClassificationEvaluator(labelCol='Y')
print('Test Area Under ROC', evaluator.evaluate(predictions))

Test Area Under ROC 0.8624405705229743


In [57]:
# Pregunta 3   (Modelo LR con ensamble de cat+num)
lista_etapas = []

# Tratamiento de la variable categórica
for cat in var_cat:
    strIdx = StringIndexer(inputCol=cat, outputCol=cat+'_index')
    encoder = OneHotEncoder(inputCol=cat+'_index',outputCol=cat+'_oneHot')
    lista_etapas += [strIdx,encoder]
    
# Tratamiento con la variable respuesta
strIdx = StringIndexer(inputCol='Churn',outputCol='Y')
lista_etapas.append(strIdx)

# Ensamblador de variables categóricas y numéricas
columnasVectores =  [c+'_oneHot' for c in var_cat] + var_num
ensamblador = VectorAssembler(inputCols=columnasVectores,outputCol='X')
lista_etapas.append(ensamblador)

procesadorEtapas = Pipeline(stages=lista_etapas)
modelo = procesadorEtapas.fit(df)
df2 = modelo.transform(df)

# Construyendo el modelo de regresión logística
train, test = df2.randomSplit([0.8, 0.2], seed = 10)
lr = LogisticRegression(featuresCol = 'X', labelCol = 'Y')
lrModel = lr.fit(train)
predictions = lrModel.transform(test)
evaluator = BinaryClassificationEvaluator(labelCol='Y')
print('Test Area Under ROC', evaluator.evaluate(predictions))

Test Area Under ROC 0.8943011093502372


In [58]:
# Pregunta 4   (Modelo LR con ensamble de cat+num y Normalización MinMax)
lista_etapas = []

# Tratamiento de la variable categórica
for cat in var_cat:
    strIdx = StringIndexer(inputCol=cat, outputCol=cat+'_index')
    encoder = OneHotEncoder(inputCol=cat+'_index',outputCol=cat+'_oneHot')
    lista_etapas += [strIdx,encoder]
    
# Tratamiento con la variable respuesta
strIdx = StringIndexer(inputCol='Churn',outputCol='Y')
lista_etapas.append(strIdx)

# Ensamblador de variables categóricas y numéricas
columnasVectores =  [c+'_oneHot' for c in var_cat] + var_num
ensamblador = VectorAssembler(inputCols=columnasVectores,outputCol='X')
lista_etapas.append(ensamblador)

# Normalización
from pyspark.ml.feature import MinMaxScaler

scaler = MinMaxScaler(inputCol='X',outputCol='X_scaled')
lista_etapas.append(scaler)

procesadorEtapas = Pipeline(stages=lista_etapas)
modelo = procesadorEtapas.fit(df)
df2 = modelo.transform(df)

# Construyendo el modelo de regresión logística
train, test = df2.randomSplit([0.8, 0.2], seed = 10)
lr = LogisticRegression(featuresCol = 'X_scaled', labelCol = 'Y')
lrModel = lr.fit(train)
predictions = lrModel.transform(test)
evaluator = BinaryClassificationEvaluator(labelCol='Y')
print('Test Area Under ROC', evaluator.evaluate(predictions))

Test Area Under ROC 0.8945039619651339


In [59]:
# Pregunta 5   (Modelo LR con ensamble de cat+num y Normalización StandardScaler)
lista_etapas = []

# Tratamiento de la variable categórica
for cat in var_cat:
    strIdx = StringIndexer(inputCol=cat, outputCol=cat+'_index')
    encoder = OneHotEncoder(inputCol=cat+'_index',outputCol=cat+'_oneHot')
    lista_etapas += [strIdx,encoder]
    
# Tratamiento con la variable respuesta
strIdx = StringIndexer(inputCol='Churn',outputCol='Y')
lista_etapas.append(strIdx)

# Ensamblador de variables categóricas y numéricas
columnasVectores =  [c+'_oneHot' for c in var_cat] + var_num
ensamblador = VectorAssembler(inputCols=columnasVectores,outputCol='X')
lista_etapas.append(ensamblador)

# Normalización
from pyspark.ml.feature import StandardScaler

scaler = MinMaxScaler(inputCol='X',outputCol='X_scaled')
lista_etapas.append(scaler)

procesadorEtapas = Pipeline(stages=lista_etapas)
modelo = procesadorEtapas.fit(df)
df2 = modelo.transform(df)

# Construyendo el modelo de regresión logística
train, test = df2.randomSplit([0.8, 0.2], seed = 10)
lr = LogisticRegression(featuresCol = 'X_scaled', labelCol = 'Y')
lrModel = lr.fit(train)
predictions = lrModel.transform(test)
evaluator = BinaryClassificationEvaluator(labelCol='Y')
print('Test Area Under ROC', evaluator.evaluate(predictions))

Test Area Under ROC 0.8945039619651339


In [ ]:
######## En este caso, para la construcción del modelo, el tipo de normalización no influye, pues se obtiene el mismo resultado

In [ ]:
# Pregunta 6   (Modelo K-means con ensamble de cat+num y Normalización StandardScaler)

In [65]:
lista_etapas = []

# Tratamiento de la variable categórica
for cat in var_cat:
    strIdx = StringIndexer(inputCol=cat, outputCol=cat+'_index')
    encoder = OneHotEncoder(inputCol=cat+'_index',outputCol=cat+'_oneHot')
    lista_etapas += [strIdx,encoder]
    
# Tratamiento con la variable respuesta
strIdx = StringIndexer(inputCol='Churn',outputCol='Y')
lista_etapas.append(strIdx)

# Ensamblador de variables categóricas y numéricas
columnasVectores =  [c+'_oneHot' for c in var_cat] + var_num
ensamblador = VectorAssembler(inputCols=columnasVectores,outputCol='X')
lista_etapas.append(ensamblador)

# Normalización
from pyspark.ml.feature import StandardScaler

scaler = MinMaxScaler(inputCol='X',outputCol='X_scaled')
lista_etapas.append(scaler)

procesadorEtapas = Pipeline(stages=lista_etapas)
modelo = procesadorEtapas.fit(df)
df2 = modelo.transform(df)

In [74]:
# Construyendo el modelo K-means
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

mejorSilhouette=20
for i in range(2,22):   # 20 valores de K 
    modelo = KMeans(k = i,featuresCol='X_scaled',predictionCol='Outputs')
    modelo = modelo.fit(df2)
    preds = modelo.transform(df2)

    # Evaluando el modelo con el K=i
    evaluator = ClusteringEvaluator(predictionCol='Outputs',featuresCol='X_scaled')
    silhouette = evaluator.evaluate(preds)
    if(silhouette<mejorSilhouette):
          = silhouette
        mejorK = i

In [79]:
print('El mejor K es:',mejorK, 'y tiene como silhouette: ',mejorSilhouette)

El mejor K es: 3 y tiene como silhouette:  0.18019185189546888
